In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jul  3 11:01:15 2023

@author: subuxin
"""
from sklearn.isotonic import isotonic_regression
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import scipy.stats as stats
from itertools import combinations
import itertools
import matplotlib.pyplot as plt
import statistics
from statsmodels.stats.contingency_tables import mcnemar
import scipy.stats as stats


# Load CSV file into a pandas DataFrame
df = pd.read_csv(r'/content/drive/MyDrive/Research/Ranking_ICML/rank_author_dec.csv')
df = df.drop_duplicates(['submission_id', 'author_id'])

# For each author, list its ranked submissions according to its ranking

author_submission_rank_old = {}

# Extract the unique authors from the DataFrame
authors = df['author_id'].unique()

# Loop through every submission
for author in authors:

    author_submission_rank_old[author] = []

    # Extract the rating as sets
    submissions = df[df['author_id'] == author]['submission_id'].tolist()

    for i in range(len(submissions)):

        rank = df[(df['submission_id'] == submissions[i]) & (df['author_id'] == author)]['rank'].tolist()
        rank = rank[0]

        # Add the ratings to the results dictionary
        author_submission_rank_old[author].append((submissions[i], rank))

def sort_submissions(author_submission_rank_old):
    for author in author_submission_rank_old:
        author_submission_rank_old[author].sort(key=lambda x: x[1], reverse=False)
    return author_submission_rank_old

author_submission_rank_old = sort_submissions(author_submission_rank_old)




df['decision'] = df['decision'].map({'Withdrawn or Desk Rejected': 0.0, 'Reject': 0.0, 'Accept (Poster)': 0.0, 'Accept (Oral & Poster)':1.0})


High_decision = []
Low_decision = []

for author in author_submission_rank_old:
    # for i in range( - 1):
      # if abs( author_submission_rank_old[author][i][2] - author_submission_rank_old[author][i+1][2] ) <= 0.25:
        decision = df[df['submission_id'] == author_submission_rank_old[author][0][0]]['decision'].tolist()
        decision = decision[0]
        # print("High:", author_submission_rank_old[author][i][0], decision)
        High_decision.append(decision)

        decision = df[df['submission_id'] == author_submission_rank_old[author][-1][0]]['decision'].tolist()
        decision = decision[0]
        # print("Low:", author_submission_rank_old[author][len(author_submission_rank_old[author])][0], decision)
        Low_decision.append(decision)


# Create a contingency table
contingency_table = [[0, 0], [0, 0]]

# Count the occurrences of the four possible outcomes
for i in range(len(High_decision)):
    if High_decision[i] == 1:
        contingency_table[0][0] += 1
    if High_decision[i] == 0:
        contingency_table[0][1] += 1
    if Low_decision[i] == 1:
        contingency_table[1][0] += 1
    if Low_decision[i] == 0:
        contingency_table[1][1] += 1


res = stats.chi2_contingency(np.array(contingency_table), correction=True, lambda_=None)

print(contingency_table)

print("high:", contingency_table[0][0]/(contingency_table[0][0] + contingency_table[0][1]))
print("low:", contingency_table[1][0]/(contingency_table[1][0] + contingency_table[1][1]))

# Print the results
print("Chi-square test statistic:", res.statistic)
print("P-value:", res.pvalue)


[[60, 1282], [30, 1312]]
high: 0.044709388971684055
low: 0.022354694485842028
Chi-square test statistic: 9.668654159170737
P-value: 0.0018743857093879365
